In [ ]:
#can use verilog internal random functions (and thus not fixing IN_WIDTH)

In [ ]:
#testbench generator MatrixMatrixMultiply_#M_#N_#Q_#prPR_#pcPC_SI_HRx#hrf_C#mamcs_A#baseAdderSize_(N)IL_(N)CL_(N)RL_(N)MR -v1.2

In [ ]:
import sys
import math
import random

In [ ]:
def MatrixMatrixMultiply_SI_testbench_F(parameters, Print_To_File, Files_Location, IN_WIDTH):

    M = parameters["M"]
    N = parameters["N"]
    Q = parameters["Q"]
    PR = parameters["PR"]
    PC = parameters["PC"]
    HRR = parameters["HRR"]
    MAMCS = parameters["MAMCS"]
    AdderSize = parameters["ADDER_SIZE"]
    Enable_Input_Latch = parameters["ENABLE_INPUT_LATCH"]
    Enable_Column_Latch = parameters["ENABLE_COLUMN_LATCH"]
    Enable_Row_Latch = parameters["ENABLE_ROW_LATCH"]
    Enable_SIM_Output_Registers = parameters["ENABLE_SIM_OUTPUT_REGISTERS"]

    Enable_Extra_Data = 1
    
    #INPUT_REG_DEPTH>=0
    INPUT_REG_DEPTH = 1
    #MULT_PIPE_DEPTH>=0
    MULT_PIPE_DEPTH = 1

    NMult = math.ceil(N/HRR)
    while math.ceil(N/NMult) != HRR:
        HRR -= 1
    RS = math.ceil(M/PR)
    CS = math.ceil(Q/PC)
    NumOfComponents = math.ceil(NMult/MAMCS)

    ModuleName = "MatrixMatrixMultiply_{}_{}_{}_{}PR_{}PC_SI_HRx{}_C{}_A{}_".format(M, N, Q, PR, PC, HRR, MAMCS, AdderSize)
    if Enable_Input_Latch<=0:
        ModuleName += "N"
    ModuleName += "IL_"
    if Enable_Column_Latch<=0:
        ModuleName += "N"
    ModuleName += "CL_"
    if Enable_Row_Latch<=0:
        ModuleName += "N"
    ModuleName += "RL_"
    if Enable_SIM_Output_Registers<=0:
        ModuleName += "N"
    ModuleName += "MR"
    UUTModuleName = ModuleName
    ModuleName += "_test_IW{}".format(IN_WIDTH)

    if Print_To_File<=0:
        of = sys.stdout
    else:
        of = open(Files_Location+UUTModuleName+"/"+ModuleName+".v", 'w+')

    print("`timescale 1ns / 500ps\n", file=of)
    print("module "+ModuleName, file=of)
    print("#(parameter", file=of)
    print("INPUT_REG_DEPTH = {},".format(INPUT_REG_DEPTH), file=of)
    print("MULT_PIPE_DEPTH = {}".format(MULT_PIPE_DEPTH), file=of)
    print(");", file=of)
    print("//IN_WIDTH fixed to {}".format(IN_WIDTH), file=of)
    print(file=of)

    print("reg clk, reset, enable;", file=of)
    print("wire newInSeriesStart;", file=of)
    lghrr=math.ceil(math.log2(HRR))
    print("wire ", end='', file=of)
    if lghrr==1:
        print("inSeries;", file=of)
    elif lghrr>1:
        print("[{}:0] inSeries;".format(lghrr-1), file=of)
    lgRS=math.ceil(math.log2(RS))
    print("wire ", end='', file=of)
    if lgRS<=1:
        print("rowSetInNo;", file=of)
    else:
        print("[{}:0] rowSetInNo;".format(lgRS-1), file=of)
    lgCS=math.ceil(math.log2(CS))
    print("wire ", end='', file=of)
    if lgCS<=1:
        print("columnSetInNo;", file=of)
    else:
        print("[{}:0] columnSetInNo;".format(lgCS-1), file=of)
    print("reg inReady;", file=of)
    print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
    for i in range(PR):
        for j in range(N):
            print("AE{}R{}".format(j, i), end='', file=of)
            if i==(PR-1) and j==(N-1):
                print(";", file=of)
            elif j!=(N-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    #         print("AE{}R{}".format(j, i), end='', file=of)
    #         if i!=(PR-1) or j!=(N-1):
    #             print(", ", end='', file=of)
    # print(";", file=of)
    print("reg signed [{}:0] ".format(IN_WIDTH-1), end='', file=of)
    for i in range(PC):
        for j in range(N):
            print("BE{}C{}".format(j, i), end='', file=of)
            if i==(PC-1) and j==(N-1):
                print(";", file=of)
            elif j!=(N-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    ac=CS*PC-Q
    QE = PC-ac-1
    if QE==0:
        stc1="C0"
    else:
        stc1="C0toC{}".format(QE)
    if QE==(PC-2):
        stc2="C{}".format(PC-1)
    else:
        stc2="C{}toC{}".format(QE+1, PC-1)
    ar=RS*PR-M
    ME = PR-ar-1
    if ME==0:
        str1="R0"
    else:
        str1="R0toR{}".format(ME)
    if ME==(PR-2):
        str2="R{}".format(PR-1)
    else:
        str2="R{}toR{}".format(ME+1, PR-1)
    print("wire "+stc1+str1+"EReady;", file=of)
    if ar!=0:
        print("wire "+stc1+str2+"EReady;", file=of)
    else:
        print("wire NUEReady1; //not used", file=of)
    if ac!=0:
        print("wire "+stc2+str1+"EReady;", file=of)
        if ar!=0:
            print("wire "+stc2+str2+"EReady;", file=of)
        else:
            print("wire NUEReady2; //not used", file=of)
    else:
        print("wire NUEReady2; //not used", file=of)
        print("wire NUEReady3; //not used", file=of)
    if lgRS<=1:
        print("wire rowSetOutNo;", file=of)
    else:
        print("wire [{}:0] rowSetOutNo;".format(lgRS-1), file=of)
    if lgCS<=1:
        print("wire columnSetOutNo;", file=of)
    else:
        print("wire [{}:0] columnSetOutNo;".format(lgCS-1), file=of)
    lgN=math.ceil(math.log2(N))
    OUT_WIDTH=2*IN_WIDTH+lgN
    #al=-1+lgN
    print("wire signed [{}:0] ".format(OUT_WIDTH-1), end='',file=of)
    for i in range(PC):
        for j in range(PR):
            print("EC{}R{}".format(i, j), end='',file=of)
            if i==(PC-1) and j==(PR-1):
                print(";", file=of)
            elif j!=(PR-1):
                print(", ", end='', file=of)            
            else:
                print(",", file=of)
    print("wire early"+stc1+str1+"EReady;", file=of)
    if ar!=0:
        print("wire early"+stc1+str2+"EReady;", file=of)
    else:
        print("wire NUeEReady1; //not used", file=of)
    if ac!=0:
        print("wire early"+stc2+str1+"EReady;", file=of)
        if ar!=0:
            print("wire early"+stc2+str2+"EReady;", file=of)
        else:
            print("wire NUeEReady2; //not used", file=of)
    else:
        print("wire NUeEReady2; //not used", file=of)
        print("wire NUeEReady3; //not used", file=of)
    print(file=of)

    print(UUTModuleName, file=of)
    print("#({}, INPUT_REG_DEPTH, MULT_PIPE_DEPTH)".format(IN_WIDTH), file=of)
    print("UUT(", file=of)

    print("clk, reset, enable,", file=of)
    print("newInSeriesStart,", file=of)
    print("inSeries,", file=of)
    print("rowSetInNo,", file=of)
    print("columnSetInNo,", file=of)
    print("inReady,", file=of)
    for i in range(PR):
        for j in range(N):
            print("AE{}R{}, ".format(j, i), end='', file=of)
        print(file=of)
    for i in range(PC):
        for j in range(N):
            print("BE{}C{}, ".format(j, i), end='', file=of)
        print(file=of)
    print(stc1+str1+"EReady,", file=of)
    if ar!=0:
        print(stc1+str2+"EReady,", file=of)
    else:
        print("NUEReady1, //not used", file=of)
    if ac!=0:
        print(stc2+str1+"EReady,", file=of)
        if ar!=0:
            print(stc2+str2+"EReady,", file=of)
        else:
            print("NUEReady2, //not used", file=of)
    else:
        print("NUEReady2, //not used", file=of)
        print("NUEReady3, //not used", file=of)
    print("rowSetOutNo,", file=of)
    print("columnSetOutNo,", file=of)
    for i in range(PC):
        for j in range(PR):
            print("EC{}R{}, ".format(i, j), end='',file=of)
        print(file=of)
    print("early"+stc1+str1+"EReady,", file=of)
    if ar!=0:
        print("early"+stc1+str2+"EReady,", file=of)
    else:
        print("NUeEReady1, //not used", file=of)
    if ac!=0:
        print("early"+stc2+str1+"EReady,", file=of)
        if ar!=0:
            print("early"+stc2+str2+"EReady", file=of)
        else:
            print("NUeEReady2 //not used", file=of)
    else:
        print("NUeEReady2, //not used", file=of)
        print("NUeEReady3 //not used", file=of)
    print(");\n", file=of)

    InputRangeStart = -2**(IN_WIDTH-1)
    InputRangeEnd = 2**(IN_WIDTH-1)
    matrix_MN = [[random.randrange(InputRangeStart, InputRangeEnd) for nn in range(N)] for jj in range(M)]
    matrix_NQ = [[random.randrange(InputRangeStart, InputRangeEnd) for ii in range(Q)] for nn in range(N)]

    # print(matrix_MN)
    # print(matrix_NQ)

    matrix_MQ = [[0 for ii in range(Q)] for jj in range(M)]
    for jj in range(M):
        for ii in range(Q):
            res=0
            for nn in range(N):
                res += matrix_MN[jj][nn]*matrix_NQ[nn][ii]
            matrix_MQ[jj][ii]=res
    # print(matrix_MQ)

    listOfAnswer = [[]for k in range(CS*RS)]

    print("initial begin", file=of)
    print("\tclk = 1;", file=of)
    print("\treset <= 0;", file=of)
    print("\tenable <= 1;", file=of)
    print("\tinReady <= 1;", file=of)

    FirstInput = True
    k=0;
    for i in range(CS):
        for j in range(RS):
            if FirstInput:
                FirstInput = False
            elif Enable_Extra_Data and Enable_Input_Latch:
                print("\t#1;", file=of)
            else:
                print("\t#{};".format(HRR), file=of)            
            #print(file=of)
            print("\t//", end='', file=of)
            if Enable_Row_Latch and i>0:
                print("[", end='', file=of)
            print("R", end='', file=of)
            for jj in range(j*PR, (j+1)*PR):
                if jj<M:
                    print(" {}".format(jj), end='', file=of)
            if Enable_Row_Latch and i>0:
                print("]", end='', file=of)
            print(", ", end='', file=of)
            if Enable_Column_Latch and j>0:
                print("[", end='', file=of)
            print("C", end='', file=of)    
            for ii in range(i*PC, (i+1)*PC):
                if ii<Q:
                    print(" {}".format(ii), end='', file=of)
            if Enable_Column_Latch and j>0:
                print("]", end='', file=of)
            print(file=of)

            for ii in range(i*PC, (i+1)*PC):
                if ii<Q:
                    for jj in range(j*PR, (j+1)*PR):
                        if jj<M:
                            listOfAnswer[k].append((ii,jj))
            k += 1;

            jc=0
            if i==0 or not Enable_Row_Latch:
                for jj in range(j*PR, (j+1)*PR):
                    #print("\tjj:{}".format(jj), file=of)
                    if jj<M:
                        for nn in range(N):
                            cMN = matrix_MN[jj][nn]
                            if cMN >= 0:
                                print("\tAE{}R{} <= {}'d{};".format(nn, jc, IN_WIDTH, cMN), file=of)
                            else:
                                print("\tAE{}R{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cMN)), file=of)                        
                        jc += 1
                    else:
                        break
            elif Enable_Extra_Data:
                print("\t//Extra Data:", file=of)
                for jj in range(j*PR, (j+1)*PR):
                    if jj<M:
                        for nn in range(N):
                            cR = random.randrange(InputRangeStart, InputRangeEnd)
                            if cR >= 0:
                                print("\tAE{}R{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                            else:
                                print("\tAE{}R{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)                        
                        jc += 1
                    else:
                        break

            ic=0
            if j==0 or not Enable_Column_Latch:
                for ii in range(i*PC, (i+1)*PC):
                    #print("\tii:{}".format(jj), file=of)
                    if ii<Q:
                        for nn in range(N):
                            cNQ = matrix_NQ[nn][ii]
                            if cNQ >= 0:
                                print("\tBE{}C{} <= {}'d{};".format(nn, ic, IN_WIDTH, cNQ), file=of)
                            else:
                                print("\tBE{}C{} <= -{}'d{};".format(nn, ic, IN_WIDTH, abs(cNQ)), file=of)                        
                        ic += 1
                    else:
                        break
            elif Enable_Extra_Data:
                print("\t//Extra Data:", file=of)
                for ii in range(i*PC, (i+1)*PC):
                    if ii<Q:
                        for nn in range(N):
                            cR = random.randrange(InputRangeStart, InputRangeEnd)
                            if cR >= 0:
                                print("\tBE{}C{} <= {}'d{};".format(nn, ic, IN_WIDTH, cR), file=of)
                            else:
                                print("\tBE{}C{} <= -{}'d{};".format(nn, ic, IN_WIDTH, abs(cR)), file=of)                        
                        ic += 1
                    else:
                        break

            if Enable_Input_Latch and Enable_Extra_Data:
                for h in range(HRR-1):
                    print("\t#1;", file=of)
                    print("\t//Extra Data:", file=of)
                    jc=0
                    ic=0
                    for jj in range(j*PR, (j+1)*PR):
                        if jj<M:
                            for nn in range(N):
                                cR = random.randrange(InputRangeStart, InputRangeEnd)
                                if cR >= 0:
                                    print("\tAE{}R{} <= {}'d{};".format(nn, jc, IN_WIDTH, cR), file=of)
                                else:
                                    print("\tAE{}R{} <= -{}'d{};".format(nn, jc, IN_WIDTH, abs(cR)), file=of)
                            jc += 1
                        else:
                            break
                    for ii in range(i*PC, (i+1)*PC):
                        if ii<Q:
                            for nn in range(N):
                                cR = random.randrange(InputRangeStart, InputRangeEnd)
                                if cR >= 0:
                                    print("\tBE{}C{} <= {}'d{};".format(nn, ic, IN_WIDTH, cR), file=of)
                                else:
                                    print("\tBE{}C{} <= -{}'d{};".format(nn, ic, IN_WIDTH, abs(cR)), file=of)                      
                            ic += 1
                        else:
                            break

    print("end", file=of)
    print(file=of)

    print("always #0.5 clk = ~ clk;", file=of)
    print(file=of)

    initial_delay_p = 0
    print("//initial delay:", file=of)
    if Enable_SIM_Output_Registers<=0:
        print("//NMR: ", end='', file=of)
        print("0", file=of)
    else:
        print("//MR: ", end='', file=of)
        print("1", file=of)
        initial_delay_p += 1
    print("//INPUT_REG_DEPTH + MULT_PIPE_DEPTH", file=of)
    print("//C{}: {}".format(MAMCS, MAMCS-1), file=of)
    initial_delay_p += (MAMCS-1)
    if AdderSize==0:
        print("//A0, Adder Tree Delay: 0", file=of)    
    else:
        AdderTreeDelay = math.ceil(math.log(NumOfComponents, AdderSize))
        print("//S{}E, C{}, NC{}, A{}, Adder Tree Delay: {}".format(NMult, MAMCS, NumOfComponents, AdderSize, AdderTreeDelay), file=of)
        initial_delay_p += AdderTreeDelay
    print("//HRx{}: {}".format(HRR, HRR), file=of)
    initial_delay_p += HRR
    print("parameter initial_delay = INPUT_REG_DEPTH + MULT_PIPE_DEPTH + {};".format(initial_delay_p), file=of)
    print(file=of)

    print("integer outfile;", file=of)
    print(file=of)

    print("task Print_Parameters_Close;", file=of)
    print("\tbegin", file=of)
    print("\t$fdisplay(outfile, \"\\nParameters' values:\");", file=of)
    print("\t$fdisplay(outfile, \"IN_WIDTH : fixed to {}\");".format(IN_WIDTH), file=of)
    print("\t$fdisplay(outfile, \"INPUT_REG_DEPTH = %0d\", INPUT_REG_DEPTH);", file=of)
    print("\t$fdisplay(outfile, \"MULT_PIPE_DEPTH = %0d\", MULT_PIPE_DEPTH);", file=of)
    print("\t$fclose(outfile);", file=of)
    print("\tend", file=of)
    print("endtask", file=of)
    print(file=of)

    print("task Simulation_Fail;", file=of)
    print("\tbegin", file=of)
    print("\t$fdisplay(outfile, \"Failed\");", file=of)
    print("\t$fdisplay(outfile, \"Error at %t\", $realtime);", file=of)
    print("\tPrint_Parameters_Close();", file=of)
    print("\t$display(\"Failed\");", file=of)
    print("\tend", file=of)
    print("endtask", file=of)
    print(file=of)

    outFileNameLen = 27+len("{}".format(IN_WIDTH))
    print("reg [{}*8-1:0] outFileName; //INPUT_REG_DEPTH and MULT_PIPE_DEPTH: 1 character".format(outFileNameLen), file=of)
    print("initial begin", file=of)
    print("$timeformat (-9, 1, \" ns\", 0);", file=of)
    print("$sformat(outFileName,\"_{}_%0d_%0d_Simulation_Result.txt\",INPUT_REG_DEPTH,MULT_PIPE_DEPTH);".format(IN_WIDTH), file=of)
    print("outfile = $fopen(outFileName, \"w\");", file=of)
    print("#initial_delay;", file=of)
    print("#0.5;", file=of)

    #print(listOfAnswer)
    FirstOutputSeries = True
    for k in range(RS*CS):
        FirstOutput = True
        if FirstOutputSeries:
            FirstOutputSeries = False
        else:
            print("#{};".format(HRR), file=of)
        print("if(", end='', file=of)
        for ans in listOfAnswer[k]:
            if FirstOutput:
                FirstOutput = False
            else:
                print(" || ", end='', file=of)
            cMQ = matrix_MQ[ans[1]][ans[0]]
            if cMQ >= 0:
                print("EC{}R{}!={}'d{}".format(ans[0]%PC, ans[1]%PR, OUT_WIDTH, cMQ), end='', file=of)
            else:
                print("EC{}R{}!=-{}'d{}".format(ans[0]%PC, ans[1]%PR, OUT_WIDTH, abs(cMQ)), end='', file=of) 
        print(") begin", file=of)
        print("\tSimulation_Fail();", file=of)
        print("\t$finish;", file=of)
        print("end", file=of)

    print("#0.5;", file=of)
    print("$fdisplay(outfile, \"Succeeded\");", file=of)
    print("$fdisplay(outfile, \"Simulation completed without error at %t\", $realtime);", file=of)
    print("Print_Parameters_Close();", file=of)
    print("$display(\"Succeeded\");", file=of)
    print("$finish;", file=of)
    print("end", file=of)
    print(file=of)
    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()